In [24]:
#r "nuget:Microsoft.ML"
#r "nuget:XPlot.Plotly"
#r "nuget:XPlot.Plotly.Interactive"
#r "nuget:Microsoft.Data.Analysis"
using System;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;

Installed Packages Microsoft.Data.Analysis, 0.18.0 Microsoft.ML, 1.6.0 XPlot.Plotly, 4.0.3 XPlot.Plotly.Interactive, 4.0.3

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [25]:
internal class DailyRecord
{
    [LoadColumn(0)] public float Number { get; set; }
    [LoadColumn(1)] public float Date { get; set; }
    [LoadColumn(2)] public float Season { get; set; }
    [LoadColumn(3)] public float Year { get; set; }
    [LoadColumn(4)] public float Month { get; set; }
    [LoadColumn(5)] public float IsHoliday { get; set; }
    [LoadColumn(6)] public float DayOfWeek { get; set; }
    [LoadColumn(7)] public float IsWorkingDay { get; set; }
    [LoadColumn(8)] public float Weather { get; set; }
    [LoadColumn(9)] public float Temperature { get; set; }
    [LoadColumn(10)] public float TemperatureFeelsLike { get; set; }
    [LoadColumn(11)] public float Humidity { get; set; }
    [LoadColumn(12)] public float Wind { get; set; }
    [LoadColumn(13)] public float Guests { get; set; }
    [LoadColumn(14)] public float Regular { get; set; }
    [LoadColumn(15)] [ColumnName("Label")] public float Total { get; set; }
}

internal class Prediction
{
    public float Score { get; set; }
}

In [26]:
DailyRecord MapToDailyRecord(string dailyInfoString)
{
    var dailyRecordInfo = dailyInfoString.Split(',', StringSplitOptions.TrimEntries);
    DateTime.TryParse(dailyRecordInfo[1], out var dateTime);
    var date = dateTime.ToOADate();
    return new DailyRecord()
    {
        Number = float.Parse(dailyRecordInfo[0]),
        Date = (float)date,
        Season = float.Parse(dailyRecordInfo[2]),
        Year = float.Parse(dailyRecordInfo[3]),
        Month = float.Parse(dailyRecordInfo[4]),
        IsHoliday = float.Parse(dailyRecordInfo[5]),
        DayOfWeek = float.Parse(dailyRecordInfo[6]),
        IsWorkingDay = float.Parse(dailyRecordInfo[7]),
        Weather = float.Parse(dailyRecordInfo[8]),
        Temperature = float.Parse(dailyRecordInfo[9]),
        TemperatureFeelsLike = float.Parse(dailyRecordInfo[10]),
        Humidity = float.Parse(dailyRecordInfo[11]),
        Wind = float.Parse(dailyRecordInfo[12]),
        Guests = float.Parse(dailyRecordInfo[13]),
        Regular = float.Parse(dailyRecordInfo[14]),
        Total = float.Parse(dailyRecordInfo[15]),
    };
}

In [27]:
var path = "bikes.csv";
var dailyRecordInfos = File.ReadLines(path)
    .Skip(1)
    .Select(MapToDailyRecord)
    .ToArray();

In [28]:
MLContext mlContext = new(20210923);

var rowsDataView = mlContext.Data.LoadFromEnumerable(dailyRecordInfos);
var shuffledData = mlContext.Data.ShuffleRows(rowsDataView, 20210923);
var trainTestData = mlContext.Data.TrainTestSplit(shuffledData, 0.2);
var trainingData = trainTestData.TrainSet;
var testData = trainTestData.TestSet;

In [29]:
var features = mlContext.Data.CreateEnumerable<DailyRecord>(trainingData, false);
display(features.Take(10));

index,Number,Date,Season,Year,Month,IsHoliday,DayOfWeek,IsWorkingDay,Weather,Temperature,TemperatureFeelsLike,Humidity,Wind,Guests,Regular,Total
0,281,40824,4,0,10,0,6,0,1,0.521667,0.513242,0.70125,0.0454042,2235,3174,5409
1,34,40577,1,0,2,0,4,1,1,0.186957,0.177878,0.437826,0.277752,61,1489,1550
2,223,40766,3,0,8,0,4,1,1,0.7175,0.651521,0.42375,0.164796,812,3980,4792
3,249,40792,3,0,9,0,2,1,3,0.54,0.5152,0.886957,0.343943,204,2506,2710
4,612,41155,3,1,9,1,1,0,1,0.7075,0.661629,0.790833,0.151121,1965,4069,6034
5,723,41266,1,1,12,0,0,0,1,0.245833,0.259471,0.515417,0.133083,408,1379,1787
6,3,40546,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
7,79,40622,1,0,3,0,0,0,1,0.3325,0.32575,0.47375,0.207721,1047,1424,2471
8,91,40634,2,0,4,0,5,1,2,0.3,0.283454,0.68625,0.258708,307,1920,2227
9,714,41257,4,1,12,0,5,1,1,0.281667,0.294192,0.642917,0.131229,429,5182,5611


In [30]:
var totals = trainingData.GetColumn<float>("Label");
var totalsHistogram = Chart.Plot(
    new Histogram { x = totals }
);
display(totalsHistogram);

<!DOCTYPE html>

In [31]:
var f1 = features.Select(f => f.Number);
var f2 = features.Select(f => f.Date);
var f3 = features.Select(f => f.Season);
var f4 = features.Select(f => f.Year);
var f5 = features.Select(f => f.Month);
var f6 = features.Select(f => f.IsHoliday);
var f7 = features.Select(f => f.DayOfWeek);
var f8 = features.Select(f => f.IsWorkingDay);
var f9 = features.Select(f => f.Weather);
var f10 = features.Select(f => f.Temperature);
var f11 = features.Select(f => f.TemperatureFeelsLike);
var f12 = features.Select(f => f.Humidity);
var f13 = features.Select(f => f.Wind);
var f14 = features.Select(f => f.Guests);
var f15 = features.Select(f => f.Regular);

var featuresDiagram = Chart.Plot(new[] {
    new Box { y = f1, name = "Number" },
    new Box { y = f2, name = "Date" },
    new Box { y = f3, name = "Season" },
    new Box { y = f4, name = "Year" },
    new Box { y = f5, name = "Month" },
    new Box { y = f6, name = "IsHoliday" },
    new Box { y = f7, name = "DayOfWeek" },
    new Box { y = f8, name = "IsWorkingDay" },
    new Box { y = f9, name = "Weather" },
    new Box { y = f10, name = "Temperature" },
    new Box { y = f11, name = "TemperatureFeelsLike" },
    new Box { y = f12, name = "Humidity" },
    new Box { y = f13, name = "Wind" },
    new Box { y = f14, name = "Guests" },
    new Box { y = f15, name = "Regular" },
});
display(featuresDiagram);

<!DOCTYPE html>

In [32]:
#r "nuget:MathNet.Numerics, 4.9.0"
using MathNet.Numerics.Statistics;

Installed Packages MathNet.Numerics, 4.9.0

In [33]:
var featureColumns = new string[] { "Number", "Date", "Season", "Year", "Month", "IsHoliday", "DayOfWeek", "IsWorkingDay", "Weather", "Temperature",
                                   "TemperatureFeelsLike", "Humidity", "Wind", "Guests", "Regular" };

var correlationMatrix = new List<List<double>>();
correlationMatrix.Add(f1.Select(x => (double)x).ToList());
correlationMatrix.Add(f2.Select(x => (double)x).ToList());
correlationMatrix.Add(f3.Select(x => (double)x).ToList());
correlationMatrix.Add(f4.Select(x => (double)x).ToList());
correlationMatrix.Add(f5.Select(x => (double)x).ToList());
correlationMatrix.Add(f6.Select(x => (double)x).ToList());
correlationMatrix.Add(f7.Select(x => (double)x).ToList());
correlationMatrix.Add(f8.Select(x => (double)x).ToList());
correlationMatrix.Add(f9.Select(x => (double)x).ToList());
correlationMatrix.Add(f10.Select(x => (double)x).ToList());
correlationMatrix.Add(f11.Select(x => (double)x).ToList());
correlationMatrix.Add(f12.Select(x => (double)x).ToList());
correlationMatrix.Add(f13.Select(x => (double)x).ToList());
correlationMatrix.Add(f14.Select(x => (double)x).ToList());
correlationMatrix.Add(f15.Select(x => (double)x).ToList());

var length = featureColumns.Length;

var z = new double[length, length];
for (int x = 0; x < length; ++x)
{
    for (int y = 0; y < length - 1 - x; ++y)
    {
        var seriesA = correlationMatrix[x];
        var seriesB = correlationMatrix[length - 1 - y];

        var value = Correlation.Pearson(seriesA, seriesB);

        z[x, y] = value;
        z[length - 1 - y, length - 1 - x] = value;
    }

    z[x, length - 1 - x] = 1;
}

var correlationMatrixHeatmap = Chart.Plot(
    new Heatmap 
    {
        x = featureColumns,
        y = featureColumns.Reverse(),
        z = z,
        zmin = -1,
        zmax = 1
    }
);
display(correlationMatrixHeatmap);

<!DOCTYPE html>